# <code>geo_bezier_3d</code>
------------------------------------
## Ahmed Body

Para construção do Ahmed Body serão usados cilindros, intersecções e superfícies de Bézier. 

Todo o processo será comentado e explicado. Caso surjam dúvidas o usuário pode conferir **Docstring** e **How To** na [documentação](https://geo-bezier-3d.readthedocs.io/). 

<img src="images/ahmed.png" width="450"/>

### 1. Primeiros passos

Bem como na documentação (How To), inicialmente o usuário deverá setar os parâmetros do domínio, dar nome ao projeto, importar bibliotecas e arquivos e setar as informações de refinamento:

In [ ]:
#lendo de outra forma: 'eixo_x'=[comprimento lx, número de nós nx, is_periodic]

domain_info={'x':[6,501,False], 
             'y':[2,201,False],
             'z':[2,201,False]}

In [2]:
for axis,(length,nodes,is_periodic) in domain_info.items():
    if is_periodic==True:
        domain_info["%s"%(axis)]+=[length/(nodes)]
    else:
        domain_info["%s"%(axis)]+=[length/(nodes-1)]

In [3]:
name = str(input('Give your project a name: ')) #nome do projeto

Give your project a name:  AHMED


In [4]:
open(f'inputs.py', 'w').close()
with open(f'inputs.py', 'a') as the_file:
    the_file.write(f'domain_info={domain_info}\n')
    the_file.write('name='),the_file.write(f'"'), the_file.write(f'{name}'), the_file.write(f'"')

In [5]:
#importação de bibliotecas e arquivos auxiliares do código

from infos import *
from inputs import *
from creating_solid import *
import inputs as i
import infos as s
import creating_solid as c

In [6]:
gen_raf_information(nraf=3) #valor de nraf, o fator que refinará a malha em cada direção.

raf=False #para construir as epsis refinadas, apenas alterar essa variável para True

### 2. Criação dos limites

Setados os parâmetros iniciais, o usuário construirá de fato o sólido:

In [7]:
#parâmetros do Ahmed Body no domínio

lenght_position=0.25*lx #a posição que o corpo ficará em relação ao comprimento

ramp_angle=25 #angulação da descontinuidade final do Ahmed Body

wide_centralize_term=((lz/2)-(1.15/2)) #nada para modificar aqui, esse termo apenas centraliza o corpo em relação ao domínio


Por facilidade, a construção do sólido será feita apenas pela metade e no final um espelhamento será feito.

Inicialmente será feita apenas a construção da parte frontal do Ahmed Body, a mais complicada (onde há arredondamentos).

Para construção dessa parte, serão usados 3 cilindros:


In [8]:
#limpando alguma possível bagunça nos dicionários auxiliares, boa prática

c.eq_storage={}
c.list_storage={}

#inicializando os 3 cilindros

gen_cylinder(identif='0', #o termo de identificação de features deve sempre começar em 0, outra boa prática
             name='down horz',
             bases_plane='xy',
             radius=0.295,
             center_1=0.295+lenght_position, #quando bases_plane equivale 'xy', center_1 está relacionado ao eixo x
             center_2=0.443, #quando bases_plane equivale 'xy', center_2 está relacionado ao eixo y
             init_height=0.00+wide_centralize_term, 
             final_height=0.575+wide_centralize_term)

gen_cylinder(identif='1',
             name='up horz',
             bases_plane='xy',
             radius=0.295,
             center_1=0.295+lenght_position,
             center_2=0.704,
             init_height=0.00+wide_centralize_term,
             final_height=0.575+wide_centralize_term)

gen_cylinder(identif='2',
             name='right vert',
             bases_plane='xz',
             radius=0.295,
             center_1=0.295+lenght_position,
             center_2=0.295+wide_centralize_term,
             init_height=0.148,
             final_height=1.00)

Para visualizar e checar se as superfícies estão como desejadas:

In [9]:
%matplotlib qt

In [10]:
#surface_plot(init_identif='3',final_identif='6',engine='matplotlib',alpha=0.5)

Aparentemente tudo está ok, embora um pouco confuso.

Depois da criação e da checagem das superfícies, vamos validá-las na matriz $\epsilon$.

Como desenvolvido na documentação (How To), para obter a intersecção entre superfícies o usuário deve setar o argumento <code>add_or_sub='add'<\code>.


In [11]:
#validação do cilindro horizontal inferior e do único cilindro vertical na epsi

gen_epsi_cylinder('0', 'solid', add_or_sub='add', cyl_raf_path=raf)
gen_epsi_cylinder('2', 'solid', add_or_sub='add', cyl_raf_path=raf)

#0: |100%|#############################################|  Elapsed Time: 0:00:00
#2: |100%|#############################################|  Elapsed Time: 0:00:00


Para efeitos de estudo a visualização da $\epsilon$ inacabada é uma boa ídeia.

O usuário deve rodar a próxima célula, sair do Jupyter, entrar no Paraview e visualizar o arquivo .xdmf com a opção XDMF Reader.

In [12]:
#gen_output(names=name,out_raf_path=raf)

O usuário consegue perceber duas cores diferentes na representação da $\epsilon$. 

Acontece que a cor mais viva é justamente a desejada, onde os dois cilindros se encontram.

Para excluir tudo o que não é a cor viva (na $\epsilon$, a cor viva representa 2), devemos rodar a função <code>normalize_epsi</code> e então gerar os arquivos de saída novamente.

In [13]:
normalize_epsi(intersection=True,target=2,epsi_raf_path=raf) #adquirindo apenas a intersecção

#gen_output(names=name,out_raf_path=raf) #gerando novamente os arquivos de saída

Novamente, pede-se para que o usuário vá ao ParaView e abra a geração 2 do arquivo .xdmf.

Para corrigir a cor, o usuário deve ir no menu **Proprieties (esquerda inferior) -> Coloring -> Rescale to data...**

<img src="images/paraview_rescale.png" width="300"/>

Agora sim a representação do que é desejado está correta.

Repetindo esse mesmo processo para a parte superior:

In [14]:
#validação do cilindro horizontal inferior e do único cilindro vertical na epsi

gen_epsi_cylinder(identif='1', surface_type='solid', add_or_sub='add', cyl_raf_path=raf)
gen_epsi_cylinder(identif='2', surface_type='solid', add_or_sub='add', cyl_raf_path=raf)

normalize_epsi(intersection=True,target=2,epsi_raf_path=raf)

#gen_output(names=name,out_raf_path=raf) #gerando novamente os arquivos de saída

#1: |100%|#############################################|  Elapsed Time: 0:00:00
#2: |100%|#############################################|  Elapsed Time: 0:00:00


Visualizando no ParaView pela 3ª vez, é possível enxergar que já se tem bom caminho andado.

Para finalizar a parte frontal, serão criados mais 3 cilindros para preencher os vazios.

Notar que dessa vez não queremos intersecções e portanto não alteraremos o argumento <code>add_or_sub</code> na hora de validar os limites na matriz! O valor default, <code>'sub'</code> já é o desejado: onde não for sólido será setado, e onde já é não sofrerá qualquer modificação. Também não precisará chamar a função de normalização da $\epsilon$, <code>normalize_epsi</code>.

In [15]:
#mais 3 cilindros

gen_cylinder(identif='3',
             name='down hor',
             bases_plane='xy',
             radius=0.295,
             center_1=0.295+lenght_position,
             center_2=0.443,
             init_height=0.295+wide_centralize_term, 
             final_height=0.575+wide_centralize_term)

gen_cylinder(identif='4',
             name='up hor',
             bases_plane='xy',
             radius=0.295,
             center_1=0.295+lenght_position,
             center_2=0.704,
             init_height=0.295+wide_centralize_term,
             final_height=0.575+wide_centralize_term)

gen_cylinder(identif='5',
             name='right vert',
             bases_plane='xz',
             radius=0.295,
             center_1=0.295+lenght_position,
             center_2=0.295+wide_centralize_term,
             init_height=0.443,
             final_height=0.704)

In [16]:
#validando esses cilindros na epsi

gen_epsi_cylinder(identif='3', surface_type='solid', cyl_raf_path=raf) #sem add_or_sub desta vez
gen_epsi_cylinder(identif='4', surface_type='solid', cyl_raf_path=raf)
gen_epsi_cylinder(identif='5', surface_type='solid', cyl_raf_path=raf)

#gerando pela 4ª vez os arquivos de saída

#gen_output(names=name,out_raf_path=raf)

#3: |100%|#############################################|  Elapsed Time: 0:00:00
#4: |100%|#############################################|  Elapsed Time: 0:00:00
#5: |100%|#############################################|  Elapsed Time: 0:00:00


Ótimo, já temos uma geometria bem interessante, visualizando no ParaView. 

Falta preencher o buraco que sobrou no meio. 

Isso será feito com um prisma quadrático:

In [17]:
#criando o preenchimento do buraco do meio com um prisma quadrático

gen_quad_prism(identif='7',
               name='filler',
               a=0.295*2,
               b=0.704-0.443,
               c=0.575-0.295,
               reference_point=[lenght_position,0.443,0.295+wide_centralize_term])

gen_epsi_quad_prism(identif='7',surface_type='solid',qp_raf_path=raf)

#gerando pela 5ª vez (!) os arquivos de saída (isso não é nem um pouco natural, só está sendo feito por conta da explicação.)

#gen_output(names=name,out_raf_path=raf)

#7: |100%|#############################################|  Elapsed Time: 0:00:00


Tudo correu bem. Visualizando no ParaView o arquivo da 5ª geração, temos exatamente metade da parte frontal do Ahmed Body.

A parte mais difícil já foi. 

Agora, será resolvida a parte do corpo do Ahmed Body.

Para tanto, será criado um grande prisma que depois será "recortado" pela superfície de Bézier que representará o ângulo da traseira:

In [18]:
#prisma quadrático do corpo:

gen_quad_prism(identif='8',
               name='body',
               a=3.088-0.295,
               b=1.000-0.147,
               c=0.575,
               reference_point=[0.295+lenght_position,0.147,wide_centralize_term])

gen_epsi_quad_prism(identif='8',surface_type='solid',qp_raf_path=raf)

#setando os cálculos com as variáveis para representar o ângulo solicitado no início do projeto:

hipo=0.656 

adj=math.cos(math.radians(ramp_angle))*hipo
opo=math.sin(math.radians(ramp_angle))*hipo

#criando a superfície de Bézier que representará a rampa traseira:

set_point_matrix(num_u_points=2,num_v_points=2)

point_storage['P00']=[(3.088-adj),1.00,0.000]
point_storage['P01']=[(3.088-adj),1.00,0.575]

point_storage['P10']=[3.088,1.00-opo,0.000]
point_storage['P11']=[3.088,1.00-opo,0.575]

create_point_matrix()

translate(direction='x',quantity=lenght_position) 
translate(direction='z',quantity=wide_centralize_term)

gen_bezier_surface(identif='9',name='ramp back')

#validando a superfície como uma saída na epsi, ou seja: 
#a partir dessa superfície (partindo da origem do plano 'zy') não existe mais sólido

gen_epsi_bezier_surface(surface_type='entry+exit and/or exit',
                        plane='zy',
                        identif='9', 
                        bez_raf_path=raf)

#gerando pela 6ª vez os arquivos de saída:

#gen_output(names=name,out_raf_path=raf)

#8: |100%|#############################################|  Elapsed Time: 0:00:00
#9 (1624 knot², order 2): |100%|#######################|  Elapsed Time: 0:00:28


Quase lá. 

Visualizando no ParaView, a única coisa que falta são as "rodas", que no Ahmed Body são representadas como pequenos cilindros:

In [19]:
#criando as 2 'rodas':

gen_cylinder(identif='10',
             name='right f wheel',
             bases_plane='xz',
             radius=0.082/2,
             center_1=0.591+lenght_position,
             center_2=0.137+wide_centralize_term,
             init_height=0.00, 
             final_height=0.147)

gen_cylinder(identif='11',
             name='right b wheel',
             bases_plane='xz',
             radius=0.082/2,
             center_1=2.145+lenght_position,
             center_2=0.137+wide_centralize_term,
             init_height=0.00,
             final_height=0.147)

#validando isso na epsi:

gen_epsi_cylinder(identif='10', surface_type='solid', cyl_raf_path=raf)
gen_epsi_cylinder(identif='11', surface_type='solid', cyl_raf_path=raf)

#quase que pela última vez, gerando os arquivos de saída:

#gen_output(names=name,out_raf_path=raf)

#10: |100%|############################################|  Elapsed Time: 0:00:00
#11: |100%|############################################|  Elapsed Time: 0:00:00


A única coisa que falta é espelhar o domínio inteiro!

In [20]:
#espelhando o domínio inteiro na direção de z:

gen_epsi_mirror(target='whole_domain',direction='z', mirror_raf_path=raf)

#finalmente gerando os últimos arquivos de saída:

gen_output(name,out_raf_path=raf)

Se o usuário seguiu exatamente o Notebook como ele foi criado, deve obter algo do tipo:
    
<img src="images/ahmed_epsi.png" width="800"/>

E o processo está finalizado! Lembre-se: o <code>Incompact3d</code> solicita as $\epsilon$ refinadas. Caso o usuário for utilizar o Ahmed Body criado para uma simulação neste solver, terá que setar o parâmetro raf (criado lá no começo, um pouco antes do início da seção 2) como <code>True</code>.